In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
m_players=pd.read_csv('/content/drive/MyDrive/male_players (legacy).csv')
players = pd.read_csv('/content/drive/MyDrive/players_22-1.csv')

<ipython-input-75-6a953d564ab5>:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  m_players=pd.read_csv('/content/drive/MyDrive/male_players (legacy).csv')
<ipython-input-75-6a953d564ab5>:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv('/content/drive/MyDrive/players_22-1.csv')


In [ ]:
m_players.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [ ]:
m_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 110 entries, player_id to player_face_url
dtypes: float64(18), int64(45), object(47)
memory usage: 135.6+ MB


In [ ]:
columns_to_drop =  [
     'player_url', 'short_name', 'long_name',
    'player_positions', 'dob', 'nationality_id', 'nationality_name',
    'body_type', 'real_face', 'player_face_url'
]
m_players = m_players.drop(columns=columns_to_drop)

In [ ]:
m_players.drop(columns=['fifa_update_date', 'player_id', 'fifa_version'], inplace=True)

In [ ]:
missing_percentage = m_players.isnull().sum() / len(m_players) * 100
drop_columns = missing_percentage[missing_percentage > 30].index
m_players = m_players.drop(drop_columns, axis=1)

In [ ]:
m_players.head()

,fifa_update,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_id,league_name,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,2,93,95,100500000.0,550000.0,27,169,67,53.0,La Liga,...,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3
1,2,92,92,79000000.0,375000.0,29,185,80,53.0,La Liga,...,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3
2,2,90,90,54500000.0,275000.0,30,180,80,19.0,Bundesliga,...,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3
3,2,90,90,52500000.0,275000.0,32,195,95,16.0,Ligue 1,...,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3
4,2,90,90,63500000.0,300000.0,28,193,92,19.0,Bundesliga,...,40+3,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3


In [ ]:
numeric_data_train=m_players.select_dtypes(include=np.number)
non_numeric_train=m_players.select_dtypes(include=['object'])

In [ ]:
numeric_imputer = SimpleImputer(strategy='mean')
m_players[numeric_data_train.columns] = numeric_imputer.fit_transform(m_players[numeric_data_train.columns])

In [ ]:
non_numeric_imputer = SimpleImputer(strategy='most_frequent')
m_players[non_numeric_train.columns] = non_numeric_imputer.fit_transform(m_players[non_numeric_train.columns])

In [ ]:
label_encoder = LabelEncoder()
for column in non_numeric_train.columns:
    # Convert the column to string type before applying LabelEncoder
    m_players[column] = m_players[column].astype(str)
    m_players[column] = label_encoder.fit_transform(m_players[column])

In [ ]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 16.1+ MB


In [ ]:
players=players.drop(columns=columns_to_drop)
players.head()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,club_name,...,lb,lcb,cb,rcb,rb,gk,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,93,93,78000000.0,320000.0,34,170,72,73.0,Paris Saint-Germain,...,61+3,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,92,92,119500000.0,270000.0,32,185,81,21.0,FC Bayern München,...,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,91,91,45000000.0,270000.0,36,187,83,11.0,Manchester United,...,60+3,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,91,91,129000000.0,270000.0,29,175,68,73.0,Paris Saint-Germain,...,62+3,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,91,91,125500000.0,350000.0,30,181,70,10.0,Manchester City,...,75+3,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [ ]:
missing_percentage = players.isnull().sum() / len(players) * 100
drop_columns = missing_percentage[missing_percentage > 30].index
players = players.drop(drop_columns, axis=1)

In [ ]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 92 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    19239 non-null  int64  
 1   overall                      19239 non-null  int64  
 2   potential                    19239 non-null  int64  
 3   value_eur                    19165 non-null  float64
 4   wage_eur                     19178 non-null  float64
 5   age                          19239 non-null  int64  
 6   height_cm                    19239 non-null  int64  
 7   weight_kg                    19239 non-null  int64  
 8   club_team_id                 19178 non-null  float64
 9   club_name                    19178 non-null  object 
 10  league_name                  19178 non-null  object 
 11  league_level                 19178 non-null  float64
 12  club_position                19178 non-null  object 
 13  club_jersey_numb

In [ ]:
numeric_data_test=players.select_dtypes(include=np.number)
non_numeric_test=players.select_dtypes(include=['object'])

In [ ]:
numeric_imputer = SimpleImputer(strategy='mean')
players[numeric_data_test.columns] = numeric_imputer.fit_transform(players[numeric_data_test.columns])

In [ ]:
non_numeric_imputer = SimpleImputer(strategy='most_frequent')
players[non_numeric_test.columns] = non_numeric_imputer.fit_transform(players[non_numeric_test.columns])

In [ ]:
X_train = m_players.drop('overall', axis=1)
y_train = m_players['overall']

In [ ]:
train_rf = RandomForestRegressor()
train_rf.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
feat_imp_train = train_rf.feature_importances_
feat_imp_df_train = pd.DataFrame({'Feature': X_train.columns, 'Importance': feat_imp_train})
feat_imp_df_train = feat_imp_df_train.sort_values(by='Importance', ascending=False)


In [ ]:
top_features_train = feat_imp_df_train.head(6)['Feature'].tolist()
top_features_train_importances = feat_imp_df_train.head(6)['Importance'].tolist()

In [ ]:
top_features_train

['value_eur',
 'age',
 'potential',
 'movement_reactions',
 'club_joined_date',
 'wage_eur']

In [ ]:
top_features_train_importances

[0.806190337212478,
 0.0764751810282518,
 0.04353732596347104,
 0.019325227799352345,
 0.014376111894931867,
 0.007308321997646465]

In [ ]:
from sklearn.preprocessing import StandardScaler
X_train = m_players[top_features_train]
y_train = m_players['overall']

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
pd.DataFrame(X_train, columns=top_features_train)

,value_eur,age,potential,movement_reactions,club_joined_date,wage_eur
0,16.451449,0.401872,3.875315,3.535775,-2.139727,24.706815
1,12.848571,0.830120,3.396013,3.099571,-1.935608,16.687273
2,8.742966,1.044244,3.076478,2.990520,-1.916313,12.104678
3,8.407815,1.472491,3.076478,2.554317,-1.519247,12.104678
4,10.251147,0.615996,3.076478,2.990520,-1.700009,13.250326
...,...,...,...,...,...,...
161578,-0.371477,-1.525244,-1.556774,-2.462022,1.814688,-0.465381
161579,-0.371477,-1.311120,-2.036076,-2.134869,1.814688,-0.463090
161580,-0.371477,-1.311120,-2.036076,-1.262462,1.647128,-0.474547
161581,-0.364774,-1.739368,-0.118868,-1.807717,1.867495,-0.474547


In [ ]:
X_test = players.drop('overall', axis=1)
Y_test = players['overall']

In [ ]:
print(X_test.columns)

Index(['sofifa_id', 'potential', 'value_eur', 'wage_eur', 'age', 'height_cm',
       'weight_kg', 'club_team_id', 'club_name', 'league_name', 'league_level',
       'club_position', 'club_jersey_number', 'club_joined',
       'club_contract_valid_until', 'preferred_foot', 'weak_foot',
       'skill_moves', 'international_reputation', 'work_rate',
       'release_clause_eur', 'pace', 'shooting', 'passing', 'dribbling',
       'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_posi

In [ ]:
# Ensure the columns are in the same order as during fitting
X_test['club_joined_date'] =0
X_test = X_test[top_features_train]

# Now try scaling again
X_test_scaled = scaler.transform(X_test)
pd.DataFrame(X_test_scaled, columns=top_features_train)

,value_eur,age,potential,movement_reactions,club_joined_date,wage_eur
0,12.680995,1.900739,3.555780,3.535775,-2.188471,14.166845
1,19.635387,1.472491,3.396013,3.426724,-2.188471,11.875548
2,7.150997,2.328987,3.236246,3.535775,-2.188471,11.875548
3,21.227356,0.830120,3.236246,2.990520,-2.188471,11.875548
4,20.640841,1.044244,3.236246,3.208622,-2.188471,15.541624
...,...,...,...,...,...,...
19234,-0.378180,-0.668748,-2.994680,-0.935310,-2.188471,-0.451634
19235,-0.371477,-1.311120,-1.876309,-1.371513,-2.188471,-0.474547
19236,-0.373153,-0.882872,-2.515378,-1.698666,-2.188471,-0.474547
19237,-0.371477,-1.311120,-1.716541,-1.480564,-2.188471,-0.474547


In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
def evaluate_model(model, X, y_true):
    y_pred = model.predict(X)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return  mae, r2

In [ ]:
rf_model = RandomForestRegressor(random_state=42, n_jobs=-1)
rf_params = {'n_estimators': [100, 200,400], 'max_depth': [10,30,40]}
rf_grid = GridSearchCV(rf_model, rf_params, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
rf_grid.fit(Xtrain, ytrain)
rf_model = rf_grid.best_estimator_
cross_val_scores = cross_val_score (rf_model, Xtrain, ytrain, cv=5, scoring='neg_mean_absolute_error')
print("Cross-Validation MAE Scores:", -cross_val_scores)
rf_metrics = evaluate_model(rf_model, Xtest, ytest)
print('\nBest RandomForestRegressor Metrics (on Test Set):', rf_metrics)
print('\n')

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Cross-Validation MAE Scores: [0.4875594  0.4884067  0.49602385 0.49080165 0.49235216]

Best RandomForestRegressor Metrics (on Test Set): (0.48187576959593226, 0.9861231920599879)




In [ ]:
xgb_model = XGBRegressor(random_state=42, n_jobs=-1)
xgb_params = {'n_estimators': [50, 100, 200], 'max_depth': [2, 10, 20], 'learning_rate': [0.1, 0.001,0.01]}
xgb_grid = GridSearchCV(xgb_model, xgb_params, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
xgb_grid.fit(Xtrain, ytrain)
xgb_model = xgb_grid.best_estimator_
cross_val_scores_xgb = cross_val_score(xgb_model, Xtrain, ytrain, cv=5, scoring='neg_mean_absolute_error')
print("Cross-Validation MAE Scores for XGBoost:", -cross_val_scores_xgb)
xgb_metrics = evaluate_model(xgb_model, Xtest, ytest)
print('\nBest XGBoost Metrics (on Test Set):', xgb_metrics)
print('\n')

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Cross-Validation MAE Scores for XGBoost: [0.50777305 0.50399946 0.51162732 0.51052833 0.50771273]

Best XGBoost Metrics (on Test Set): (0.4999261085219167, 0.9851143174678495)




In [ ]:
gb_model = GradientBoostingRegressor(random_state=42)
gb_params = {'n_estimators': [50, 100, 200], 'max_depth': [2, 10, 20],'learning_rate': [0.1, 0.01,0.01]}
gb_grid = GridSearchCV(gb_model, gb_params, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
gb_grid.fit(Xtrain, ytrain)
gb_model = gb_grid.best_estimator_
cross_val_scores_gb = cross_val_score(gb_model, Xtrain, ytrain, cv=5, scoring='neg_mean_absolute_error')
print("Cross-Validation MAE Scores for Gradient Boosting:", -cross_val_scores_gb)
gb_metrics = evaluate_model(gb_model, Xtest, ytest)
print('\nBest Gradient Boosting Metrics (on Test Set):', gb_metrics)
print('\n')

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Cross-Validation MAE Scores for Gradient Boosting: [0.51461576 0.51312733 0.52227479 0.51768953 0.51488607]

Best Gradient Boosting Metrics (on Test Set): (0.5104423417675354, 0.9866706200221993)




In [ ]:
from sklearn.model_selection import GridSearchCV

ensemble_model = VotingRegressor(estimators=[('rf', rf_model), ('xgb', xgb_model), ('gb', gb_model)], n_jobs=-1)
ensemble_params = {
    'weights': [[1, 1, 1], [2, 1, 1], [1, 2, 1], [1, 1, 2], [2, 2, 1], [2, 1, 2], [1, 2, 2]]
    }
grid_search_ensemble = GridSearchCV(ensemble_model, ensemble_params, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search_ensemble.fit(Xtrain, ytrain)
print("Best Parameters for VotingRegressor:", grid_search_ensemble.best_params_)
print("Best Cross-Validation MAE Score for VotingRegressor:", -grid_search_ensemble.best_score_)
best_ensemble_model = grid_search_ensemble.best_estimator_
ensemble_metrics_test = evaluate_model(best_ensemble_model, Xtest, ytest)
print('VotingRegressor Test Metrics:', ensemble_metrics_test)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Parameters for VotingRegressor: {'weights': [2, 1, 1]}
Best Cross-Validation MAE Score for VotingRegressor: 0.4868093416854748
VotingRegressor Test Metrics: (0.47853500282797456, 0.9868389970569249)


In [ ]:
rf_player22_metrics = evaluate_model(rf_model, X_test, Y_test)
print('RandomForestRegressor Metrics on Player 22:', rf_player22_metrics)
print('\n')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


RandomForestRegressor Metrics on Player 22: (28.002818493684696, -16.56608791367288)




In [ ]:
xgb_player22_metrics = evaluate_model(xgb_model, X_test_scaled, Y_test)
print('XGBoost Metrics on Player 22:', xgb_player22_metrics)
print('\n')

XGBoost Metrics on Player 22: (1.863838300294022, 0.8302858718896854)




In [ ]:
gb_player22_metrics = evaluate_model(gb_model, X_test, Y_test)
print('GradientBoostingRegressor Metrics on Player 22:', gb_player22_metrics)
print('\n')

GradientBoostingRegressor Metrics on Player 22: (28.18493587662493, -16.782264944165064)




/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [ ]:
ensemble_player22_metrics = evaluate_model(best_ensemble_model, X_test_scaled, Y_test)
print('VotingRegressor Metrics on Player 22:', ensemble_player22_metrics)
print('\n')

VotingRegressor Metrics on Player 22: (1.7543320889556389, 0.8553911042048045)




In [ ]:
rf_predictions = rf_model.predict(X_test_scaled)
xgb_predictions = xgb_model.predict(X_test_scaled)
gb_predictions = gb_model.predict(X_test_scaled)
ensemble_predictions = best_ensemble_model.predict(X_test_scaled)

In [ ]:
rf_metrics_test = evaluate_model(rf_model, X_test_scaled, Y_test)
xgb_metrics_test = evaluate_model(xgb_model, X_test_scaled, Y_test)
gb_metrics_test = evaluate_model(gb_model, X_test_scaled, Y_test)
ensemble_metrics_test = evaluate_model(best_ensemble_model, X_test_scaled, Y_test)

print('RandomForestRegressor Test Metrics (player 22):', rf_metrics_test)
print ("  ")
print('XGBoost Test Metrics(player 22):', xgb_metrics_test)
print ("  ")
print('GradientBoostingRegressor Test Metrics(player 22):', gb_metrics_test)
print ("  ")
print('VotingRegressor Test Metrics:(player 22)', ensemble_metrics_test)

RandomForestRegressor Test Metrics (player 22): (1.7598805419732733, 0.8535714966509833)
  
XGBoost Test Metrics(player 22): (1.863838300294022, 0.8302858718896854)
  
GradientBoostingRegressor Test Metrics(player 22): (1.6708873042353447, 0.8737391391286315)
  
VotingRegressor Test Metrics:(player 22) (1.7543320889556389, 0.8553911042048045)


In [ ]:
import joblib
joblib.dump(rf_model, 'Fifaprediction.pkl')

['Fifaprediction.pkl']

In [ ]:
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [ ]:
joblib.dump(scaler, 'Scaler.pkl')

['Scaler.pkl']